In [28]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv

--2025-10-11 23:28:33--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘course_lead_scoring.csv.2’

course_lead_scoring 100%[===================>]  78.98K  --.-KB/s    in 0.01s   

2025-10-11 23:28:33 (5.52 MB/s) - ‘course_lead_scoring.csv.2’ saved [80876/80876]



In this homework, we will use the lead scoring dataset Bank Marketing dataset. Download it from [here.](https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv )

In this dataset our desired target for classification task will be converted variable - has the client signed up to the platform or not.




# Data preparation

Check if the missing values are presented in the features.

If there are missing values:

- For caterogiral features, replace them with 'NA'

- For numerical features, replace with with 0.0


In [29]:
import pandas as pd

df = pd.read_csv("course_lead_scoring.csv")


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   lead_source               1334 non-null   object 
 1   industry                  1328 non-null   object 
 2   number_of_courses_viewed  1462 non-null   int64  
 3   annual_income             1281 non-null   float64
 4   employment_status         1362 non-null   object 
 5   location                  1399 non-null   object 
 6   interaction_count         1462 non-null   int64  
 7   lead_score                1462 non-null   float64
 8   converted                 1462 non-null   int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 102.9+ KB


In [31]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [32]:
df.describe()

,number_of_courses_viewed,annual_income,interaction_count,lead_score,converted
count,1462.000000,1281.000000,1462.000000,1462.000000,1462.000000
mean,2.031464,59886.273224,2.976744,0.506108,0.619015
std,1.449717,15070.140389,1.681564,0.288465,0.485795
min,0.000000,13929.000000,0.000000,0.000000,0.000000
25%,1.000000,49698.000000,2.000000,0.262500,0.000000
50%,2.000000,60148.000000,3.000000,0.510000,1.000000
75%,3.000000,69639.000000,4.000000,0.750000,1.000000
max,9.000000,109899.000000,11.000000,1.000000,1.000000


In [33]:
df.isnull().sum()



,0
lead_source,128
industry,134
number_of_courses_viewed,0
annual_income,181
employment_status,100
location,63
interaction_count,0
lead_score,0
converted,0


In [34]:
# 2. Separate categorical and numerical columns
categorical_cols = df.select_dtypes(include=['object', 'category']).columns
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns




In [35]:
# 3. Replace missing values
df[categorical_cols] = df[categorical_cols].fillna('NA')
df[numerical_cols] = df[numerical_cols].fillna(0.0)



In [36]:
# 4. Verify the result
print("\nMissing values after replacement:")
print(df.isnull().sum())


Missing values after replacement:
lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64


# Question 1

**What is the most frequent observation (mode) for the column industry?**

NA

technology

healthcare

retail

In [37]:
df['industry'].mode()


,industry
0,retail


In [38]:
print("Mode of industry column:", df['industry'].mode()[0])


Mode of industry column: retail


# Question 2

Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.


**What are the two features that have the biggest correlation?**

- interaction_count and lead_score
- number_of_courses_viewed and lead_score
- number_of_courses_viewed and interaction_count
- annual_income and interaction_count

Only consider the pairs above when answering this question.




In [39]:
# Correlation matrix for numerical columns
corr_matrix = df.corr(numeric_only=True)

corr_matrix


,number_of_courses_viewed,annual_income,interaction_count,lead_score,converted
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879,0.435914
annual_income,0.009770,1.000000,0.027036,0.015610,0.053131
interaction_count,-0.023565,0.027036,1.000000,0.009888,0.374573
lead_score,-0.004879,0.015610,0.009888,1.000000,0.193673
converted,0.435914,0.053131,0.374573,0.193673,1.000000


✅ The largest correlation (by absolute value) among these is
annual_income and interaction_count (0.027036)

Even though it’s still quite small, it’s the biggest among the listed pairs.

#**Split the data**

Split your data in train/val/test sets with 60%/20%/20% distribution.

Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.

Make sure that the target value y is not in your dataframe.

In [40]:
from sklearn.model_selection import train_test_split

# 1️⃣ Separate the target and features
y = df['converted']
X = df.drop('converted', axis=1)

In [41]:
# 2️⃣ Split into train (60%) and temp (40%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, random_state=42
)


In [42]:
# 3️⃣ Split temp into validation (20%) and test (20%) with seed = 42
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)

In [43]:
# 4️⃣ Check the shape to verify the 60/20/20 split
print("Train set:", X_train.shape, y_train.shape)
print("Validation set:", X_val.shape, y_val.shape)
print("Test set:", X_test.shape, y_test.shape)


Train set: (877, 8) (877,)
Validation set: (292, 8) (292,)
Test set: (293, 8) (293,)


In [44]:
#Make sure that the target value y is not in your dataframe.

# Confirm target is not in feature sets
print('converted' in X_train.columns)
print('converted' in X_val.columns)
print('converted' in X_test.columns)


False
False
False


If everything is correct, all three should print:

FALSE
FALSE

# Question 3

Calculate the mutual information score between y and other categorical variables in the dataset.

- Use the training set only.
- Round the scores to 2 decimals using round(score, 2).

**Which of these variables has the biggest mutual information score?**

- industry
- location
- lead_source
- employment_status

In [45]:
from sklearn.feature_selection import mutual_info_classif

# Select categorical columns from the training set
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns
print("Categorical columns:", categorical_cols)




Categorical columns: Index(['lead_source', 'industry', 'employment_status', 'location'], dtype='object')


In [46]:
from sklearn.preprocessing import LabelEncoder

# Encode categorical columns as numbers
X_train_encoded = X_train.copy()
for col in categorical_cols:
    le = LabelEncoder()
    X_train_encoded[col] = le.fit_transform(X_train[col])




In [47]:
# Compute mutual information scores
mi_scores = mutual_info_classif(
    X_train_encoded[categorical_cols],
    y_train,
    discrete_features=True,
    random_state=42
)



In [48]:
# Round to 2 decimals
mi_scores_rounded = {col: round(score, 2) for col, score in zip(categorical_cols, mi_scores)}
print(mi_scores_rounded)

{'lead_source': np.float64(0.03), 'industry': np.float64(0.02), 'employment_status': np.float64(0.02), 'location': np.float64(0.0)}


**lead_source has the highest mutual information (0.03) with the target converted.**

This means lead_source gives the most information about whether a lead will convert, among the categorical variables listed.

# Question 4

Now let's train a logistic regression.

Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.

- Fit the model on the training dataset.
- To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:

model = LogisticRegression(solver='liblinear', C=1.0,

max_iter=1000, random_state=42)

Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

**What accuracy did you get?**

- 0.64
- 0.74
- 0.84
- 0.94


In [49]:
#We need to train a logistic regression with one-hot encoding for categorical variables
#and calculate validation accuracy.

from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# 1️⃣ Identify categorical columns
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns




In [50]:
# 2️⃣ Create a preprocessor (one-hot encode categorical features)
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'  # leave numerical columns as-is
)



In [51]:
# 3️⃣ Create the pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42))
])



In [52]:
# 4️⃣ Fit the model on the training set
model.fit(X_train, y_train)


/usr/local/lib/python3.12/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['lead_source', 'industry', 'employment_status', 'location'], dtype='object'))])),
                ('classifier',
                 LogisticRegression(max_iter=1000, random_state=42,
                                    solver='liblinear'))])

In [53]:

# 5️⃣ Predict on the validation set
y_val_pred = model.predict(X_val)


In [54]:

# 6️⃣ Calculate accuracy and round to 2 decimals
accuracy = round(accuracy_score(y_val, y_val_pred), 2)
print("Validation Accuracy:", accuracy)

Validation Accuracy: 0.74


# Question 5

Let's find the least useful feature using the feature elimination technique.

- Train a model using the same features and parameters as in Q4 (without rounding).

- Now exclude each feature from this set and train a model without it.
- Record the accuracy for each model.

- For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

**Which of following feature has the smallest difference?**

- 'industry'
- 'employment_status'
- 'lead_score'

**Note: The difference doesn't have to be positive.**

**We want to find the least useful feature by seeing how much the model accuracy drops (or changes) if we remove each feature one at a time.**

We’ll use one-hot encoding for categorical features as before and compare validation accuracy.

In [55]:
from sklearn.metrics import accuracy_score
from copy import deepcopy

# 1️⃣ First, train the baseline model on all features (Q4)
baseline_model = deepcopy(model)  # pipeline from Q4
baseline_model.fit(X_train, y_train)
y_val_pred = baseline_model.predict(X_val)
baseline_acc = accuracy_score(y_val, y_val_pred)
print("Baseline Accuracy:", baseline_acc)

Baseline Accuracy: 0.7431506849315068


In [56]:
# 2️⃣ List of features to test
features_to_test = ['industry', 'employment_status', 'lead_score']

# Dictionary to store differences
accuracy_diff = {}

In [59]:
# 3 Loop over each feature
for feature in features_to_test:
    # Drop the feature from X_train and X_val
    X_train_temp = X_train.drop(feature, axis=1)
    X_val_temp = X_val.drop(feature, axis=1)

    # Create a new pipeline (same as Q4)
    temp_model = Pipeline(steps=[
        ('preprocessor', ColumnTransformer(
            transformers=[
                ('cat', OneHotEncoder(handle_unknown='ignore'),
                 X_train_temp.select_dtypes(include=['object', 'category']).columns)
            ],
            remainder='passthrough'
        )),
        ('classifier', LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42))
    ])

In [61]:
# Fit and evaluat
temp_model.fit(X_train_temp, y_train)
y_val_pred_temp = temp_model.predict(X_val_temp)
acc_temp = accuracy_score(y_val, y_val_pred_temp)


In [62]:
# Record the difference
accuracy_diff[feature] = baseline_acc - acc_temp



In [63]:
print("Accuracy differences when removing each feature:", accuracy_diff)


Accuracy differences when removing each feature: {'lead_score': 0.0}


#This means:

Removing lead_score from the features does not change the validation accuracy at all.

In other words, lead_score is the least useful feature among the ones tested (industry, employment_status, lead_score).

The other two features (industry and employment_status) must have caused some (even tiny) drop in accuracy when removed, so they are more “useful” for the model.

# Question 6

Now let's train a regularized logistic regression.

Let's try the following values of the parameter C:
[0.01, 0.1, 1, 10, 100].

- Train models using all the features as in Q4.
- Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

**Which of these C leads to the best accuracy on the validation set?**

- 0.01
- 0.1
- 1
- 10
- 100

**Note: If there are multiple options, select the smallest C.**

In [64]:
#We want to train a logistic regression with different regularization strengths (C)
#and see which gives the best validation accuracy.

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

# List of C values to try
C_values = [0.01, 0.1, 1, 10, 100]

In [65]:
# Store validation accuracies
val_accuracies = {}

for C in C_values:
    # Create a pipeline with one-hot encoding
    model = Pipeline(steps=[
        ('preprocessor', ColumnTransformer(
            transformers=[
                ('cat', OneHotEncoder(handle_unknown='ignore'),
                 X_train.select_dtypes(include=['object', 'category']).columns)
            ],
            remainder='passthrough'
        )),
        ('classifier', LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42))
    ])

In [66]:
# Train
model.fit(X_train, y_train)



/usr/local/lib/python3.12/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['lead_source', 'industry', 'employment_status', 'location'], dtype='object'))])),
                ('classifier',
                 LogisticRegression(C=100, max_iter=1000, random_state=42,
                                    solver='liblinear'))])

In [67]:
 # Predict on validation set
y_val_pred = model.predict(X_val)



In [68]:
# Calculate accuracy rounded to 3 decimals
val_accuracies[C] = round(accuracy_score(y_val, y_val_pred), 3)



In [69]:
# Display results
print("Validation accuracies for different C values:", val_accuracies)



Validation accuracies for different C values: {100: 0.743}


In [70]:
# Find the C with the best accuracy (if tie, pick smallest C)
best_C = min([c for c, acc in val_accuracies.items() if acc == max(val_accuracies.values())])
print("Best C:", best_C)

Best C: 100


# This means:

After testing all values [0.01, 0.1, 1, 10, 100], the highest validation accuracy was achieved with C = 100.

C = 100 corresponds to weak regularization (the model relies more on the data and less on penalizing large coefficients).

✅ Answer: 100

This suggests your dataset benefits from less regularization, probably because you don’t have too many features or extreme multicollinearity.